In [19]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [20]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 're_uuid=4a73cfef-17cb-4bf1-b895-f538c107cc1c; reese84=3:y6KsYwsXsetcHeamprarhA==:ARvXYH5ZDPFUmZ7zYx2qdtclSOMvWDBE8nsaCb/GFSwfKZXDBNEYrjHlX5/p+tpRP85oDL5nrZc0XnuzqhTLLTf03wTm9Tj0ryfo5WYwqWVhJGvMP4eEtVL3ze7cwVM/T1VvvM0wQUEY0HV2TsLQ0YjV3DoxiPedWZZ7Ux8Sx8ic1jOOfJOETUxKmYp33vJsFkPyy2X38Rg+WiurHoUV/8RaXSIns4PA/HxAlCLWIRevIoxatW4Q8L1+dPeCK1iGQhzbBFuH40k7vc6lxnkNM3RrYtUIUIDFDrRA2Wo8Y5+CtVMbFcw+6Mw99wH2eqpFKsy6da5dWyZw8mG2O6S9DIdTghqb+1Hy5f7SBgoWlqOoXtIPJyVHOtluZup0Ilkc9FUC2fwQbOrkxy1ibjmOs/kH3xKwuzTM5WA68q6eLOYalYbrogcqL5pkJtjcLCJkPwEO5BLkylulCTsUuU5F1Q==:LeqjhgOiYn1LYE9fHBq3OZO6qIfQTvHREYcM6ZoqVXk=; didomi_token=eyJ1c2VyX2lkIjoiMTkzNzdhMjEtOTVjYy02YmQ3LWFkMTYtMTlhMWI2MDFhMTc3IiwiY3JlYXRlZCI6IjIwMjQtMTEtMjlUMTE6MTU6MzIuMzE2WiIsInVwZGF0ZWQiOiIyMDI0LTExLTI5VDExOjE1OjM5LjM2MVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQI2igAQI2igAAHABBENBRFsAP_gAEPgAAiQKoNX_G__bWlr8X73aftkeY1P9_h77sQxBhfJE-4FzLvW_JwXx2ExNA36tqIKmRIAu3TBIQNlGJDURVCgaogVryDMaEyUoTNKJ6BkiFMRM2dYCFxvm4tj-QCY5vr991dx2B-t7dr83dzyy4xHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F_pE2_eT1l_tWvp7D9-cts__XW89_fff_9Pn_-uB_-_3_vf_BU4Akw0KiAMsCQkINAwggQAqCsICKBAEAACQNEBACYMCnYGAC6wkQAgBQADBACAAEGQAIAAAIAEIgAgAKBAABAIFAAGABAMBAAQMAAIALAQCAAEB0DFMCCAQLABIzIiFMCEIBIICWyoQSAIEFcIQizwCIBETBQAAAAAFIAAgLBYHEkgJUJBAFxBtAAAQAIBBAAUIJOTAAEAZstQeDBtGVpgGD5gkQ0wDIAiCMgAAAAA.f_wACHwAAAAA; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; kanirante-adv-aauid=1917394c-2c9f-42bc-bad9-45948864e1da; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20057%7CMCMID%7C85815951856949542982648077952583171318%7CMCAID%7CNONE%7CMCOPTOUT-1732886139s%7CNONE%7CvVersion%7C4.6.0',
    "Referer": "https://www.idealista.com/alquiler-viviendas/murcia-murcia/",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
}

In [28]:
# Crear una sesión
session = requests.Session()
session.headers.update(headers)

# URL base de Fotocasa
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Paginación y almacenamiento de resultados
page = 1
all_listings = []

while True:
    try:
        print(f"Procesando página {page}...")
        # Realizar la solicitud a la página actual
        response = session.get(f"{base_url}l/{page}")
        response.raise_for_status()  # Verificar que la solicitud sea exitosa

        # Analizar el contenido HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todas las tarjetas de inmuebles
        listings = soup.find_all("article", class_="re-CardPackPremium")

        # Si no hay más tarjetas, salir del bucle
        if not listings:
            print("No se encontraron más inmuebles. Fin de la paginación.")
            break

        # Almacenar las tarjetas
        for listing in listings:
            try:
                link_tag = listing.find('a', class_='re-CardPackPremium-carousel')
                href = link_tag.get('href') if link_tag else None
                full_link = f"https://www.fotocasa.es{href}" if href and href.startswith("/") else None

                titulo = listing.find("span", class_="re-I18nPropertyTitle").get_text()
                precio = listing.find('span', class_="re-CardPrice").get_text(strip=True)

                features = listing.find("ul", class_="re-CardFeaturesWithIcons-wrapper")
                habitaciones = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--rooms").get_text(strip=True) if features else "N/A"
                banos = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--bathrooms").get_text(strip=True) if features else "N/A"
                superficie = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--surface").get_text(strip=True) if features else "N/A"
                planta = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--floor").get_text(strip=True) if features else "N/A"

                # Guardar datos del listado
                all_listings.append({
                    "Título": titulo,
                    "Precio": precio,
                    "Habitaciones": habitaciones,
                    "Baños": banos,
                    "Superficie": superficie,
                    "Planta": planta,
                    "Enlace": full_link,
                })

            except Exception as e:
                print(f"Error procesando un listado: {e}")

        # Pasar a la siguiente página
        page += 1

    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Mostrar todas las tarjetas encontradas
print(f"Se encontraron {len(all_listings)} inmuebles.")
for listing in all_listings:
    print(listing)

Procesando página 1...
Procesando página 2...
No se encontraron más inmuebles. Fin de la paginación.
Se encontraron 2 inmuebles.
{'Título': 'Apartamento con terraza en Calle NUEVO PUERTO BELLO-MANGA, Entremares ', 'Precio': '650 €', 'Habitaciones': '2 habs.', 'Baños': '1 baño', 'Superficie': '51 m²', 'Planta': '2ª Planta', 'Enlace': 'https://www.fotocasa.es/es/alquiler/vivienda/la-manga-del-mar-menor/aire-acondicionado-calefaccion-jardin-terraza-piscina/184947088/d?from=list'}
{'Título': 'Piso con balcón en Casco Antiguo ', 'Precio': '800 €', 'Habitaciones': '2 habs.', 'Baños': '1 baño', 'Superficie': '87 m²', 'Planta': '4ª Planta', 'Enlace': 'https://www.fotocasa.es/es/alquiler/vivienda/cartagena/aire-acondicionado-calefaccion-parking-ascensor/184987623/d?from=list'}
